<a href="https://colab.research.google.com/github/sultanasabiha/RAG/blob/main/RAGBackend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU\
    llama-index\
    llama_index-embeddings-huggingface\
    pinecone[grpc]\
    llama_index-vector_stores-pinecone\
    transformers\
    accelerate\
    bitsandbytes\
    llama_index-llms-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 402.8/402.8 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255

In [2]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [3]:
PC_API_KEY=userdata.get('PC_API_KEY')

The default chunk size is 1024, while the default chunk overlap is 20.
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core import Settings

documents = SimpleDirectoryReader("./data").load_data()

Settings.chunk_size = 512
Settings.chunk_overlap = 50

index = VectorStoreIndex.from_documents(
    documents,
)

query_engine = index.as_query_engine(similarity_top_k=4)

In [4]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import Settings
Settings.chunk_size = 512
Settings.chunk_overlap = 20
documents=SimpleDirectoryReader(input_files=["/content/drive/MyDrive/Colab Notebooks/Datasets/IndianFinancialNews_modified.csv"]).load_data()


In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model=HuggingFaceEmbedding()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
from pinecone.grpc import PineconeGRPC
from pinecone import ServerlessSpec
from llama_index.vector_stores.pinecone import PineconeVectorStore

In [7]:
pc=PineconeGRPC(api_key=PC_API_KEY)
index_name="rag-application"

In [8]:
pc.delete_index(index_name)


In [9]:
import time

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)



In [10]:
index = pc.Index(index_name)
vector_store = PineconeVectorStore(pinecone_index=index)

In [11]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}

In [12]:
from llama_index.core import StorageContext
context=StorageContext.from_defaults(vector_store=vector_store)

In [13]:
from llama_index.core import VectorStoreIndex
vs_index=VectorStoreIndex.from_documents(documents,storage_context=context,embed_model=embed_model)

Upserted vectors:   0%|          | 0/1386 [00:00<?, ?it/s]

In [14]:
import torch
from transformers import BitsAndBytesConfig


# quantize to save memory
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [15]:
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import logging

logging.set_verbosity_error()

llm = HuggingFaceLLM(
    model_name="mistralai/Mistral-7B-Instruct-v0.3",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.3",
    context_window=3900,
    max_new_tokens=1024,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs={"temperature": 0.7, "top_k": 5, "top_p": 0.95,"do_sample":True},
    device_map="auto",
)


/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [16]:
chat_engine=vs_index.as_chat_engine(chat_mode="context",llm=llm)

In [17]:
while(True):
  print("\n\nPrompt:")
  query=input()
  if(query=="exit"):
    print("\n\nExiting...")
    break

  response=chat_engine.stream_chat(query)

  print("\nResponse:")
  response.print_response_stream()




Prompt:
what new policies are adopted due to Covid 19?

Response:
Due to the Covid-19 pandemic, several new policies have been adopted by the Reserve Bank of India (RBI) to support the financial sector in India:

1. The RBI has reduced the reverse repo rate by 25 basis points on April 17, 2020, which is aimed at increasing liquidity in the market. (Row 147)
2. The RBI has made it easier for Non-Banking Financial Companies (NBFCs) and Microfinance Institutions (MFIs) amid the Covid-19 pandemic. (Row 145)
3. The RBI has introduced a moratorium on bank loan repayments for a certain period to help businesses and individuals affected by the pandemic. (Rows 62, 63)
4. The RBI has allowed the restructuring of loans for NBFCs, but has asked rating agencies not to downgrade these loans. (Row 65)
5. The RBI has cancelled the license of CKP Co-operative Bank and notified the cancellation on its website. (Row 67)

These policies are aimed at providing relief to various sectors of the economy, pa